In [1]:
import os
import time
import math
import numpy as np
import numpy.linalg as npla
import scipy
from scipy import sparse
from scipy import linalg
import scipy.sparse.linalg as spla
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d
#%matplotlib tk

In [2]:
def LUfactor(A, pivoting = True):
    """Factor a square matrix with partial pivoting, A[p,:] == L @ U
    Parameters: 
      A: the matrix.
      pivoting = True: whether or not to do partial pivoting
    Outputs (in order):
      L: the lower triangular factor, same dimensions as A, with ones on the diagonal
      U: the upper triangular factor, same dimensions as A
      p: the permutation vector that permutes the rows of A by partial pivoting
    """
    # Check the input
    m, n = A.shape
    assert m == n, 'input matrix A must be square'
    
    # Initialize p to be the identity permutation
    p = np.array(range(n))
    
    # Make a copy of the matrix that we will transform into L and U
    LU = A.astype(np.float64).copy()
    
    # Eliminate each column in turn
    for piv_col in range(n):
        
        # Choose the pivot row and swap it into place
        if pivoting:
            piv_row = piv_col + np.argmax(np.abs(LU[piv_col:, piv_col]))
            assert LU[piv_row, piv_col] != 0., "can't find nonzero pivot, matrix is singular"
            LU[[piv_col, piv_row], :]  = LU[[piv_row, piv_col], :]
            p[[piv_col, piv_row]]      = p[[piv_row, piv_col]]
            
        # Update the rest of the matrix
        pivot = LU[piv_col, piv_col]
        assert pivot != 0., "pivot is zero, can't continue"
        for row in range(piv_col + 1, n):
            multiplier = LU[row, piv_col] / pivot
            LU[row, piv_col] = multiplier
            LU[row, (piv_col+1):] -= multiplier * LU[piv_col, (piv_col+1):]
            
    # Separate L and U in the result
    U = np.triu(LU)
    L = LU - U + np.eye(n)
    
    return (L, U, p)

In [3]:
def Lsolve(L, b, unit_diag = False):
    """Forward solve a unit lower triangular system Ly = b for y
    Parameters: 
      L: the matrix, must be square, lower triangular, with ones on the diagonal
      b: the right-hand side vector
        unit_diag = False: if true, assume the diagonal is all ones
    Output:
      y: the solution vector to L @ y == b
    """
    # Check the input
    m, n = L.shape
    assert m == n, "matrix must be square"
    assert np.all(np.tril(L) == L), "matrix L must be lower triangular"
    if unit_diag:
        assert np.all(np.diag(L) == 1), "matrix L must have ones on the diagonal"
    bn, = b.shape
    assert bn == n, "rhs vector must be same size as L"

    # Make a copy of b that we will transform into the solution
    y = b.astype(np.float64).copy()
    
    # Forward solve
    for col in range(n):
        if not unit_diag:
            y[col] /= L[col, col]
        y[col+1:] -= y[col] * L[col+1:, col]
        
    return y

In [4]:
def Usolve(U, y, unit_diag = False):
    """Backward solve an upper triangular system Ux = y for x
    Parameters: 
      U: the matrix, must be square, upper triangular, with nonzeros on the diagonal
      y: the right-hand side vector
      unit_diag = False: if true, assume the diagonal is all ones
    Output:
      x: the solution vector to U @ x == y
    """
    # Check the input
    m, n = U.shape
    assert m == n, "matrix must be square"
    assert np.all(np.triu(U) == U), "matrix U must be upper triangular"
    if unit_diag:
        assert np.all(np.diag(U) == 1), "matrix U must have ones on the diagonal"
    yn, = y.shape
    assert yn == n, "rhs vector must be same size as U"
    
    # Make a copy of y that we will transform into the solution
    x = y.astype(np.float64).copy()
    
    # Back solve
    for col in reversed(range(n)):
        if not unit_diag:
            x[col] /= U[col, col]
        x[:col] -= x[col] * U[:col, col]
        
    return x

In [5]:
def LUsolve(A, b):
    """Solve a linear system Ax = b for x by LU factorization with partial pivoting.
    Parameters: 
      A: the matrix.
      b: the right-hand side
    Outputs (in order):
      x: the computed solution
      rel_res: relative residual norm,
        norm(b - Ax) / norm(b)
    """
    # Check the input
    m, n = A.shape
    assert m == n, "matrix must be square"
    bn, = b.shape
    assert bn == n, "rhs vector must be same size as matrix"
    
    # LU factorization
    L, U, p = LUfactor(A)
    
    # Forward and back substitution
    y = Lsolve(L,b[p])
    x = Usolve(U,y)
    
    # Residual norm
    rel_res = npla.norm(b - A@x) / npla.norm(b)
    
    return (x, rel_res)

In [6]:
a = np.zeros((13, 13)).astype('float64')

In [7]:
A = 1/math.sqrt(2)

In [8]:
a[1,0]=1

In [9]:
a[5,0]=-1#first

In [10]:
a[2,1]=1#second

In [11]:
a[0,2]=A

In [12]:
a[3,2]=-1

In [13]:
a[4,2]=-A#third

In [14]:
a[0,3]=A

In [15]:
a[2,3]=1

In [16]:
a[4,3]=A#forth

In [17]:
a[3,4]=1

In [18]:
a[7,4]=-1#fifth

In [19]:
a[6,5]=1#sixth

In [20]:
a[4,6]=A

In [21]:
a[5,6]=1

In [22]:
a[8,6]=-A

In [23]:
a[9,6]=-1#seventh

In [24]:
a[4,7]=A

In [25]:
a[6,7]=1

In [26]:
a[8,7]=A#eighth

In [27]:
a[9,8]=1

In [28]:
a[12,8]=-1#ninth

In [29]:
a[10,9]=1#tenth

In [30]:
a[7,10]=1

In [31]:
a[8,10]=A

In [32]:
a[11,10]=-A#elevth

In [33]:
a[8,11]=A

In [34]:
a[10,11]=1

In [35]:
a[11,11]=A#telveth

In [36]:
a[12,12]=1

In [37]:
a[11,12]=A

In [79]:
a = 1.0/math.sqrt(2)

In [80]:
b=np.array([0,10,0,0,0,0,0,15,0,20,0,0,0])

In [82]:
A = np.array([[0,1,0,0,0,-1,0,0,0,0,0,0,0],
[0,0,1,0,0,0,0,0,0,0,0,0,0],
[a,0,0,-1,-a,0,0,0,0,0,0,0,0],
[a,0,1,0,a,0,0,0,0,0,0,0,0],
[0,0,0,1,0,0,0,-1,0,0,0,0,0],
[0,0,0,0,0,0,1,0,0,0,0,0,0],
[0,0,0,0,a,1,0,0,-a,-1,0,0,0],
[0,0,0,0,a,0,1,0,a,0,0,0,0],
[0,0,0,0,0,0,0,0,0,1,0,0,-1],
[0,0,0,0,0,0,0,0,0,0,1,0,0],
[0,0,0,0,0,0,0,1,a,0,0,-a,0],
[0,0,0,0,0,0,0,0,a,0,1,a,0],
[0,0,0,0,0,0,0,0,0,0,0,a,1]
])

In [84]:
f=npla.solve(A,b)

In [85]:
f=np.round(f,3)

In [86]:
f

array([-28.284,  20.   ,  10.   , -30.   ,  14.142,  20.   ,   0.   ,
       -30.   ,   7.071,  25.   ,  20.   , -35.355,  25.   ])

array([[ 0.        ,  0.        ,  0.70710678,  0.70710678,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ],
       [ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ],
       [ 0.        ,  1.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        , -1.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        , -0.70710678,  0.70710678,  0.        ,
         0.        ,  0.70710678,  0.70710678,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ],
       [-1.        ,  0.      

In [69]:
def p5(power):
    a = 10 ** power
    A = np.array([[a,1],[1,1]])
    b=np.array([a+2,3])
    LU1 = LUfactor(A, True)
    LU2 = LUfactor(A, False)
    y1 = Lsolve(LU1[0], b[LU1[2]])
    y2 = Lsolve(LU2[0], b[LU2[2]])
    x1 = Usolve(LU1[1], y1)
    x2 = Usolve(LU2[1], y2)
    print("when alpha = 10 to the", power, ":\n","with pivoting x = ", x1,"\n without pivoting x=", x2,"\n")

In [70]:
for power in [-4,-8,-16,-20]:
    p5(power)

when alpha = 10 to the -4 :
 with pivoting x =  [1. 2.] 
 without pivoting x= [1. 2.] 

when alpha = 10 to the -8 :
 with pivoting x =  [1. 2.] 
 without pivoting x= [0.99999999 2.        ] 

when alpha = 10 to the -16 :
 with pivoting x =  [1. 2.] 
 without pivoting x= [4.4408921 2.       ] 

when alpha = 10 to the -20 :
 with pivoting x =  [1. 2.] 
 without pivoting x= [0. 2.] 



In [ ]:
#when pivoting is true, the result is more accurate no matter 
#how alpha change since it enhance numerical stability. 
#However, when pivoting is false, the answer start to fluctuate
#as alpha get smaller because numerical stability is no ensured

In [93]:
A = np.round(20*np.random.rand(6,6))

In [94]:
b = np.round(np.random.rand(6))

In [95]:
L,U,p=LUfactor(A)

In [96]:
x=Usolve(L.T, Lsolve(U.T,b[p]))

In [97]:
X=npla.solve(A.T,b)

In [98]:
res = npla.norm(X-x)

In [99]:
res

1.086857477370672

In [100]:
print(x)

[-0.17992086 -0.23880419  0.28772624 -0.04790427  0.58822703 -0.57798051]


In [101]:
print(X)

[ 0.06273283  0.27721502 -0.40341735 -0.14725657  0.19434723 -0.11601635]


In [102]:
x=LUsolve(A.T,b)

In [103]:
x

(array([ 0.06273283,  0.27721502, -0.40341735, -0.14725657,  0.19434723,
        -0.11601635]), 1.1035677597380545e-15)

In [134]:
np.round(L@U)

array([[19.,  5.,  8., 13.,  2., 11.],
       [ 9., 15.,  0., 12.,  4., 17.],
       [ 1., 12., 15., 14., 16., 17.],
       [13., 16., 12.,  0., 15.,  6.],
       [12.,  6.,  1.,  2., 16., 12.],
       [ 2.,  3.,  5., -0., 19.,  8.]])

In [138]:
A

array([[13., 16., 12.,  0., 15.,  6.],
       [12.,  6.,  1.,  2., 16., 12.],
       [ 2.,  3.,  5.,  0., 19.,  8.],
       [ 9., 15.,  0., 12.,  4., 17.],
       [ 1., 12., 15., 14., 16., 17.],
       [19.,  5.,  8., 13.,  2., 11.]])

In [144]:
P=np.array([[0,0,0,0,0,1],
           [0,0,0,1,0,0],
           [0,0,0,0,1,0],
           [1,0,0,0,0,0],
           [0,1,0,0,0,0],
           [0,0,1,0,0,0]])

In [145]:
P@A

array([[19.,  5.,  8., 13.,  2., 11.],
       [ 9., 15.,  0., 12.,  4., 17.],
       [20., 17., 23., 27., 18., 28.],
       [13., 16., 12.,  0., 15.,  6.],
       [12.,  6.,  1.,  2., 16., 12.],
       [ 2.,  3.,  5.,  0., 19.,  8.]])

In [173]:
p = np.array([5,3,4,0,1,2])

In [174]:
def getPermutation(p):
    P= np.zeros((len(p),len(p))).astype('int64')
    k = 0
    for i in p:
        P[k,i] = 1;
        k = k+1
    return P

In [175]:
P = getPermutation(p)

In [176]:
P

array([[0, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0]])

In [172]:
for i in p:
    print(i)

[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]


In [177]:
A = np.round(10*np.random.rand(6,6))

In [178]:
b=np.round(10*np.random.rand(6))

In [179]:
L,U,p = LUfactor(A)

In [203]:
P = getPermutation(p)

In [180]:
LT = L.T

In [210]:
UT = U.T

In [211]:
Y = Lsolve(UT, b)

In [212]:
X = Usolve(LT, Y)

In [213]:
x = P.T@X

In [214]:
print(x)

[-0.42505133  0.38788501  0.35954825 -0.14045175  0.88480493 -0.20821355]


In [215]:
print(npla.solve(A.T,b))

[-0.42505133  0.38788501  0.35954825 -0.14045175  0.88480493 -0.20821355]
